In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import joblib
import time
from sklearn import preprocessing

In [88]:
#Parameters initial
REGION = 25
NUM_estimators = [50]#[1,5,10,15,20,25,30,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135,140,145,150]
num_features = 3


In [89]:
#Read in file
file = pd.read_csv('con_1.csv',header = None)
feature_names = ['sensor1','sensor2','sensor3','Region','Location_x','Location_y','time']
file.columns = feature_names

In [90]:
##devide to four regions
def region_4(Y_x,Y_y):
    y_region = []
    for i in range (0,len(Y_y)):
        if Y_x[i] <= 115 and Y_y[i]<=115:
            y_region.append(0)
        if Y_x[i] <= 115 and Y_y[i]>115:
            y_region.append(1)
        if Y_x[i] > 115 and Y_y[i]<=115:
            y_region.append(2)
        if Y_x[i] > 115 and Y_y[i]>115:
            y_region.append(3)
    return y_region

In [91]:
#feature & labels
Y_region = file['Region']
Y_region = [int(i) for i in Y_region]
Y_y = file['Location_y'].values
Y_x = file['Location_x'].values
X = file[['sensor1','sensor2','sensor3']].values
if REGION == 4:
    Y_region = region_4(Y_x,Y_y)

# Random spliting by 70%/30%
(X_train,X_test,Y_region_train,Y_region_test)=train_test_split(X, Y_region,test_size=0.3)   


In [92]:
Tr_acc = []
Te_acc = []
for num_estimators in NUM_estimators:
    
    #fit into model
    Bagged_region = BaggingClassifier(n_estimators=num_estimators ,max_features=num_features,  warm_start=False, random_state=0)
    Bagged_region.fit(X_train, Y_region_train)

    #testing
    Y_region_pred_test = Bagged_region.predict(X_test)
    Y_region_pred_test= [int (i) for i in Y_region_pred_test]

    Y_region_pred_train = Bagged_region.predict(X_train)
    Y_region_pred_train= [int (i) for i in Y_region_pred_train]

    accuracy_test = 0
    accuracy_train = 0

    #Calculate for accuracy
    for i in range(0,len(Y_region_pred_train)):
        if Y_region_pred_train[i] == Y_region_train[i]:
            accuracy_train += 1
    accuracy_train = accuracy_train/len(Y_region_pred_train)*100

    for i in range(0,len(Y_region_pred_test)):
        if Y_region_pred_test[i] == Y_region_test[i]:
            accuracy_test += 1
    accuracy_test = accuracy_test/len(Y_region_pred_test)*100

    print("Region TRAIN Accuracy is %.2f"%accuracy_train, "%")
    print("Region TEST Accuracy is %.2f"%accuracy_test, "%")
    
    
    Tr_acc.append(accuracy_train)
    Te_acc.append(accuracy_test)

Region TRAIN Accuracy is 69.48 %
Region TEST Accuracy is 49.99 %


In [93]:
#Tuning for num_estimators
#Tr_acc = np.array(Tr_acc)
#Te_acc = np.array(Te_acc)
#NUM_estimators = np.array(NUM_estimators)
#plt.plot(NUM_estimators.reshape(30,1),Tr_acc.reshape(30,1),label='Train')
#plt.plot(NUM_estimators.reshape(30,1),Te_acc.reshape(30,1),label='Test')
#plt.show()

In [94]:
#save model
joblib.dump(Bagged_region, 'Bagged_region_25.model')


['Bagged_region_25.model']